<a href="https://colab.research.google.com/github/echodpp/DRL_Application/blob/main/final/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import numpy as np
import json
import random
import argparse

In [2]:
rseed = 114514
random.seed(rseed)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def read_json(file):
    with open(file) as f:
        return json.load(f)

def write_json(data, file):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

data_ml_100k = read_json("/content/drive/MyDrive/Final_Project/ml_100k.json")

In [5]:
print (data_ml_100k[0][0])
print (data_ml_100k[0][1])
print (len(data_ml_100k))

My Best Friend's Wedding | The English Patient | Face/Off | Psycho | The Princess Bride | Kids | Kids in the Hall: Brain Candy | Scream | The Craft | The Rock | The Shining | Mission: Impossible | Reality Bites | Aliens | Indiana Jones and the Last Crusade | One Flew Over the Cuckoo's Nest | In the Name of the Father | Return of the Jedi | The Silence of the Lambs | The Usual Suspects | Dances with Wolves | Monty Python's Life of Brian | Monty Python and the Holy Grail | The Full Monty | Clockwork Orange, A | Apocalypse Now | Trainspotting | The Doors | Mr. Holland's Opus | Independence Day (ID4) | Liar Liar | Titanic | What's Eating Gilbert Grape | The Saint | Donnie Brasco | The Empire Strikes Back | Beautiful Girls | Mars Attacks! | Broken Arrow | Amistad | The Long Kiss Goodnight | French Kiss | The Maltese Falcon | Dazed and Confused | Strange Days
Starship Troopers
943


In [11]:
u_item_dict = {}
u_item_p = 0
for elem in data_ml_100k:
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in u_item_dict:
            u_item_dict[movie] = u_item_p
            u_item_p +=1
print (len(u_item_dict))
u_item_len = len(u_item_dict)

user_list = []
for i, elem in  enumerate(data_ml_100k):
    item_hot_list = [0 for ii in range(u_item_len)]
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        item_pos = u_item_dict[movie]
        item_hot_list[item_pos] = 1
    user_list.append(item_hot_list)
user_matrix = np.array(user_list)
user_matrix_sim = np.dot(user_matrix, user_matrix.transpose())


pop_dict = {}
for elem in data_ml_100k:
    # elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in pop_dict:
              pop_dict[movie] = 0
        pop_dict[movie] += 1



i_item_dict = {}
i_item_id_list = []
i_item_user_dict = {}
i_item_p = 0
for i, elem in  enumerate(data_ml_100k):
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in i_item_user_dict:
            item_hot_list = [0. for ii in range(len(data_ml_100k))]
            i_item_user_dict[movie] = item_hot_list
            i_item_dict[movie] = i_item_p
            i_item_id_list.append(movie)
            i_item_p+=1
#         item_pos = item_dict[movie]
        i_item_user_dict[movie][i] += 1
#     user_list.append(item_hot_list)
i_item_s_list = []
for item in i_item_id_list:
    i_item_s_list.append(i_item_user_dict[item])
#     print (sum(item_user_dict[item]))
item_matrix = np.array(i_item_s_list)
item_matrix_sim = np.dot(item_matrix, item_matrix.transpose())

id_list =list(range(0,len(data_ml_100k)))

1493


In [13]:
def sort_uf_items(target_seq, us, num_u, num_i):

    candidate_movies_dict = {}
    sorted_us = sorted(list(enumerate(us)), key=lambda x: x[-1], reverse=True)[:num_u]
    dvd = sum([e[-1] for e in sorted_us])
    for us_i, us_v in sorted_us:
        us_w = us_v * 1.0/dvd
#         print (us_i)
        us_elem = data_ml_100k[us_i]
#         print (us_elem[0])
#         assert 1==0
        us_seq_list = us_elem[0].split(' | ')#+[us_elem[1]]

        for us_m in us_seq_list:
#             print (f"{us_m} not in {target_seq}, {us_m not in target_seq}")
#             break

            if us_m not in target_seq:
                if us_m not in candidate_movies_dict:
                    candidate_movies_dict[us_m] = 0.
                candidate_movies_dict[us_m]+=us_w

#         assert 1==0

    candidate_pairs = list(sorted(candidate_movies_dict.items(), key=lambda x:x[-1], reverse=True))
#     print (candidate_pairs)
    candidate_items = [e[0] for e in candidate_pairs][:num_i]
    return candidate_items

In [14]:
results_data_15 = []
length_limit = 8
num_u= 12
total_i = 19
count = 0
total = 0
cand_ids = []
for i in id_list[:1000]:
    elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')

    candidate_items = sort_uf_items(seq_list, user_matrix_sim[i], num_u=num_u, num_i=total_i)

    if elem[-1] in candidate_items:
#         print ('HIT: 1')
        count += 1
        cand_ids.append(i)
    else:
        pass
#         print ('HIT: 0')
    total +=1
print (f'count/total:{count}/{total}={count*1.0/total}')
print ('-----------------\n')

count/total:170/943=0.18027571580063625
-----------------



### Baseline most popular

In [9]:
import collections

# Aggregate movie popularity
def aggregate_movie_popularity(data_ml_100k):
    movie_counts = collections.Counter()
    for user_data in data_ml_100k:
        movies_watched = user_data[0].split(' | ')
        movie_counts.update(movies_watched)
    return movie_counts

def get_top_movies(movie_counts, top_n=10):
    return [movie for movie, count in movie_counts.most_common(top_n)]
# Calculate hit rate for the baseline model
def calculate_hit_rate_for_selected_users(data_ml_100k, cand_ids, top_movies):
    count = 0
    total = 0
    for i in cand_ids:
        user_data = data_ml_100k[i]
        next_watched_movie = user_data[1]
        if next_watched_movie in top_movies:
            count += 1
        total += 1
    return count, total, count / total if total else 0

In [15]:
movie_counts = aggregate_movie_popularity(data_ml_100k)

# Get the top 10 most popular movies
top_movies = get_top_movies(movie_counts)

# Calculate the hit rate for the baseline model
count, total, hit_rate = calculate_hit_rate_for_selected_users(data_ml_100k, cand_ids, top_movies)
print(f'Hit Rate for Selected Users: {hit_rate} ({count}/{total})')

Hit Rate for Selected Users: 0.11176470588235295 (19/170)


## Advanced Recommenders
### User-Based Collaborative Filtering

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Create a user-movie matrix
def create_user_movie_matrix(data_ml_100k):
    # Assuming that data_ml_100k is a list of tuples/lists in the format: [watched movies list, next watched movie]
    all_movies = set()
    for user_data in data_ml_100k:
        all_movies.update(user_data[0].split(' | '))
    all_movies = list(all_movies)
    movie_index = {movie: i for i, movie in enumerate(all_movies)}

    user_movie_matrix = np.zeros((len(data_ml_100k), len(all_movies)))
    for i, user_data in enumerate(data_ml_100k):
        for movie in user_data[0].split(' | '):
            user_movie_matrix[i, movie_index[movie]] = 1
    return user_movie_matrix, movie_index

# Recommend movies using User-Based Collaborative Filtering
def recommend_movies_user_based(target_user_index, user_movie_matrix, top_n=10):
    # Calculate similarity with other users
    similarities = cosine_similarity(user_movie_matrix[target_user_index].reshape(1, -1), user_movie_matrix)[0]
    similar_users = np.argsort(-similarities)[1:]  # Exclude the user itself

    # Aggregate watched movies from similar users
    recommended_movies_scores = np.sum(user_movie_matrix[similar_users], axis=0)

    # Filter out movies already watched by the target user
    watched_movies = user_movie_matrix[target_user_index] == 1
    recommended_movies_scores[watched_movies] = 0

    # Select top N recommendations
    recommended_movie_indices = np.argsort(-recommended_movies_scores)[:top_n]
    recommended_movies = [list(movie_index.keys())[i] for i in recommended_movie_indices]
    return recommended_movies

In [33]:
def recommend_movies_user_based_weighted(target_user_index, user_movie_matrix, top_n=10):
    # Calculate similarity with other users
    similarities = cosine_similarity(user_movie_matrix[target_user_index].reshape(1, -1), user_movie_matrix)[0]

    # Weighted sum of movie ratings across all users
    weighted_movie_scores = np.dot(similarities, user_movie_matrix)

    # Filter out movies already watched by the target user
    watched_movies = user_movie_matrix[target_user_index] == 1
    weighted_movie_scores[watched_movies] = 0

    # Select top N recommendations
    recommended_movie_indices = np.argsort(-weighted_movie_scores)[:top_n]
    recommended_movies = [list(movie_index.keys())[i] for i in recommended_movie_indices]
    return recommended_movies

In [34]:
user_movie_matrix, movie_index = create_user_movie_matrix(data_ml_100k)

In [35]:
recommended_movies = recommend_movies_user_based_weighted(0, user_movie_matrix)
print("Recommended Movies:", recommended_movies)

Recommended Movies: ['Star Wars', 'Air Force One', 'Contact', 'Fargo', 'Jerry Maguire', 'Toy Story', 'The Godfather', 'Twelve Monkeys', 'Star Trek: First Contact', 'Men in Black']


In [40]:
# Recommend movies for a list of users
def recommend_movies_for_selected_users_weighted(user_indices, user_movie_matrix, top_n=10):
    recommendations = {}
    for user_index in user_indices:
        recommended_movies = recommend_movies_user_based_weighted(user_index, user_movie_matrix, top_n)
        recommendations[user_index] = recommended_movies
    return recommendations

# Calculate hit rate
def calculate_hit_rate_for_selected_users(data_ml_100k, recommendations):
    count = 0
    total = len(recommendations)
    for user_index, recommended_movies in recommendations.items():
        next_watched_movie = data_ml_100k[user_index][1]
        if next_watched_movie in recommended_movies:
            count += 1
    return count, total, count / total if total else 0


In [32]:
# Get recommendations for selected users
recommendations = recommend_movies_for_selected_users(cand_ids, user_movie_matrix)

# Calculate the hit rate for the selected users
count, total, hit_rate = calculate_hit_rate_for_selected_users(data_ml_100k, recommendations)
print(f'Hit Rate for Selected Users: {hit_rate} ({count}/{total})')

Hit Rate for Selected Users: 0.1588235294117647 (27/170)


In [42]:
recommendations_weighted = recommend_movies_for_selected_users_weighted(cand_ids, user_movie_matrix)

# Calculate the hit rate for the selected users with weighted recommendations
count_weighted, total_weighted, hit_rate_weighted = calculate_hit_rate_for_selected_users(data_ml_100k, recommendations_weighted)
print(f'Hit Rate for Selected Users with Weighted Recommendations: {hit_rate_weighted} ({count_weighted}/{total_weighted})')

Hit Rate for Selected Users with Weighted Recommendations: 0.38235294117647056 (65/170)
